In [107]:
import pandas as pd
import numpy as np
import urllib2
import re
import requests
import json
import time
from BeautifulSoup import BeautifulSoup

In [3]:
podcastDf = pd.read_pickle('pkl/english_podcasts.pkl')

In [41]:
# small sample for testing
podcastTest = podcastDf.sample(n = 20)

In [12]:
itunesUrl = 'https://itunes.apple.com/search?'
parameters = {}
parameters['entity'] = 'podcast'
parameters['attribute'] = 'titleTerm'

requestString = ''
for param in parameters:
    paramString = param + '=' + parameters[param]
    if len(requestString) == 0:
        requestString = paramString
    else:
        requestString = requestString + '&' + paramString

In [25]:
def convert_unicode(input):
    if isinstance(input, dict):
        return {convert_unicode(key): convert_unicode(value) for key, value in input.iteritems()}
    elif isinstance(input, list):
        return [convert_unicode(element) for element in input]
    elif isinstance(input, unicode):
        return input.encode('utf-8')
    else:
        return input

In [62]:
itunes_collection_id = []
mean_rating = []
num_ratings = []

scrapeStem = 'https://itunes.apple.com/us/podcast/id'

for ind, row in podcastDf.iterrows():
    # itunes search request
    requestUrl = itunesUrl + requestString + '&term=' + urllib2.quote(row['name'])
    r = requests.get(requestUrl)
    
    # check for http errors
    try:
        r.raise_for_status()
    except r.exceptions.HTTPError as e:
        print "HTTPError:", e.message
        break
    
    # get the results
    data = r.json()
    data = data['results']
    
    # skip if no matches or >1 match
    if len(data) != 1:
        itunes_collection_id.append(np.nan)
        mean_rating.append(np.nan)
        num_ratings.append(np.nan)
        continue
    
    # scrape itunes page for ratings
    data = data[0]
    data = convert_unicode(data)
    itunes_collection_id.append(data['collectionId'])
    scrapeUrl = scrapeStem + str(data['collectionId'])
    soup = BeautifulSoup(urllib2.urlopen(scrapeUrl).read())
    thisMean = convert_unicode(re.findall(r'Customer Ratings(\d\.\d+)', soup.text))
    thisTotal = convert_unicode(re.findall(r'&nbsp;(\d+) Ratings', soup.text))
    mean_rating.append(thisMean)
    num_ratings.append(thisTotal)
    
    
        

ConnectionError: ('Connection aborted.', error(54, 'Connection reset by peer'))

In [77]:
podcast_data1 = podcastDf[:len(mean_rating)]

In [82]:
podcast_data1 = podcast_data1.assign(itunes_collection_id = itunes_collection_id,
                    mean_rating = mean_rating,
                    num_ratings = num_ratings)

In [84]:
podcast_data1.to_pickle('pkl/itunes_podcasts_pt1.pkl')

In [93]:
podcastDf2 = podcastDf.iloc[len(mean_rating):]

In [95]:
podcastDf2.shape

(46283, 13)

In [96]:
itunes_collection_id = []
mean_rating = []
num_ratings = []

scrapeStem = 'https://itunes.apple.com/us/podcast/id'

for ind, row in podcastDf2.iterrows():
    # itunes search request
    requestUrl = itunesUrl + requestString + '&term=' + urllib2.quote(row['name'])
    r = requests.get(requestUrl)
    
    # check for http errors
    try:
        r.raise_for_status()
    except r.exceptions.HTTPError as e:
        print "HTTPError:", e.message
        break
    
    # get the results
    data = r.json()
    data = data['results']
    
    # skip if no matches or >1 match
    if len(data) != 1:
        itunes_collection_id.append(np.nan)
        mean_rating.append(np.nan)
        num_ratings.append(np.nan)
        continue
    
    # scrape itunes page for ratings
    data = data[0]
    data = convert_unicode(data)
    itunes_collection_id.append(data['collectionId'])
    scrapeUrl = scrapeStem + str(data['collectionId'])
    soup = BeautifulSoup(urllib2.urlopen(scrapeUrl).read())
    thisMean = convert_unicode(re.findall(r'Customer Ratings(\d\.\d+)', soup.text))
    thisTotal = convert_unicode(re.findall(r'&nbsp;(\d+) Ratings', soup.text))
    mean_rating.append(thisMean)
    num_ratings.append(thisTotal)

HTTPError: HTTP Error 503: Service Unavailable

In [103]:
podcast_data2 = podcastDf2[:len(mean_rating)]
podcast_data2 = podcast_data2.assign(itunes_collection_id = itunes_collection_id[:len(mean_rating)],
                    mean_rating = mean_rating,
                    num_ratings = num_ratings)
podcast_data2.to_pickle('pkl/itunes_podcasts_pt2.pkl')
podcastDf3 = podcastDf2.iloc[len(mean_rating):]

In [104]:
itunes_collection_id = []
mean_rating = []
num_ratings = []

scrapeStem = 'https://itunes.apple.com/us/podcast/id'

for ind, row in podcastDf3.iterrows():
    # itunes search request
    requestUrl = itunesUrl + requestString + '&term=' + urllib2.quote(row['name'])
    
    #Retry opening url 4 times before failing, sleep 60 sec in between
    for x in range(0, 4):  # try 4 times
        try:
            urllib2.urlopen(requestUrl)
            str_error = None
        except urllib2.HTTPError as str_error:
            pass

        if str_error:
            sleep(60)  # wait for 60 seconds before trying to fetch the data again
        else:
            break
    
        
        r = requests.get(requestUrl)
    
    # get the results
    data = r.json()
    data = data['results']
    
    # skip if no matches or >1 match
    if len(data) != 1:
        itunes_collection_id.append(np.nan)
        mean_rating.append(np.nan)
        num_ratings.append(np.nan)
        continue
    
    # scrape itunes page for ratings
    data = data[0]
    data = convert_unicode(data)
    itunes_collection_id.append(data['collectionId'])
    scrapeUrl = scrapeStem + str(data['collectionId'])
    soup = BeautifulSoup(urllib2.urlopen(scrapeUrl).read())
    thisMean = convert_unicode(re.findall(r'Customer Ratings(\d\.\d+)', soup.text))
    thisTotal = convert_unicode(re.findall(r'&nbsp;(\d+) Ratings', soup.text))
    mean_rating.append(thisMean)
    num_ratings.append(thisTotal)

HTTPError: HTTP Error 503: Service Unavailable

In [105]:
len(mean_rating)

2835

In [106]:
podcast_data3 = podcastDf3[:len(mean_rating)]
podcast_data3 = podcast_data3.assign(itunes_collection_id = itunes_collection_id[:len(mean_rating)],
                    mean_rating = mean_rating,
                    num_ratings = num_ratings)
podcast_data3.to_pickle('pkl/itunes_podcasts_pt3.pkl')
podcastDf4 = podcastDf2.iloc[len(mean_rating):]

In [108]:
itunes_collection_id = []
mean_rating = []
num_ratings = []

scrapeStem = 'https://itunes.apple.com/us/podcast/id'

for ind, row in podcastDf4.iterrows():
    # itunes search request
    requestUrl = itunesUrl + requestString + '&term=' + urllib2.quote(row['name'])
    
    #Retry opening url 4 times before failing, sleep 60 sec in between
    for x in range(0, 4):  # try 4 times
        try:
            urllib2.urlopen(requestUrl)
            str_error = None
        except urllib2.HTTPError as str_error:
            pass

        if str_error:
            sleep(60)  # wait for 60 seconds before trying to fetch the data again
        else:
            break
    
        
        r = requests.get(requestUrl)
    
    # get the results
    data = r.json()
    data = data['results']
    
    # skip if no matches or >1 match
    if len(data) != 1:
        itunes_collection_id.append(np.nan)
        mean_rating.append(np.nan)
        num_ratings.append(np.nan)
        continue
    
    # scrape itunes page for ratings
    data = data[0]
    data = convert_unicode(data)
    itunes_collection_id.append(data['collectionId'])
    scrapeUrl = scrapeStem + str(data['collectionId'])
    soup = BeautifulSoup(urllib2.urlopen(scrapeUrl).read())
    thisMean = convert_unicode(re.findall(r'Customer Ratings(\d\.\d+)', soup.text))
    thisTotal = convert_unicode(re.findall(r'&nbsp;(\d+) Ratings', soup.text))
    mean_rating.append(thisMean)
    num_ratings.append(thisTotal)

URLError: <urlopen error [Errno 60] Operation timed out>

In [109]:
len(mean_rating)

23064

In [110]:
podcast_data4 = podcastDf4[:len(mean_rating)]
podcast_data4 = podcast_data4.assign(itunes_collection_id = itunes_collection_id[:len(mean_rating)],
                    mean_rating = mean_rating,
                    num_ratings = num_ratings)
podcast_data4.to_pickle('pkl/itunes_podcasts_pt4.pkl')
podcastDf5 = podcastDf4.iloc[len(mean_rating):]

In [ ]:
itunes_collection_id = []
mean_rating = []
num_ratings = []

scrapeStem = 'https://itunes.apple.com/us/podcast/id'

for ind, row in podcastDf5.iterrows():
    # itunes search request
    requestUrl = itunesUrl + requestString + '&term=' + urllib2.quote(row['name'])
    
    #Retry opening url 4 times before failing, sleep 60 sec in between
    for x in range(0, 4):  # try 4 times
        try:
            urllib2.urlopen(requestUrl)
            str_error = None
        except urllib2.HTTPError as str_error:
            pass

        if str_error:
            sleep(60)  # wait for 60 seconds before trying to fetch the data again
        else:
            break
    
        
        r = requests.get(requestUrl)
    
    # get the results
    data = r.json()
    data = data['results']
    
    # skip if no matches or >1 match
    if len(data) != 1:
        itunes_collection_id.append(np.nan)
        mean_rating.append(np.nan)
        num_ratings.append(np.nan)
        continue
    
    # scrape itunes page for ratings
    data = data[0]
    data = convert_unicode(data)
    itunes_collection_id.append(data['collectionId'])
    scrapeUrl = scrapeStem + str(data['collectionId'])
    soup = BeautifulSoup(urllib2.urlopen(scrapeUrl).read())
    thisMean = convert_unicode(re.findall(r'Customer Ratings(\d\.\d+)', soup.text))
    thisTotal = convert_unicode(re.findall(r'&nbsp;(\d+) Ratings', soup.text))
    mean_rating.append(thisMean)
    num_ratings.append(thisTotal)